Mariana MacDonald
Week 5

Chapters 5 and 6

Chapter 5- Exercise 5.1

**Exercise:** In the BRFSS (see Section 5.4), the distribution of heights is roughly normal with parameters µ = 178 cm and σ = 7.7 cm for men, and µ = 163 cm and σ = 7.3 cm for women.

In order to join Blue Man Group, you have to be male between 5’10” and 6’1” (see http://bluemancasting.com). What percentage of the U.S. male population is in this range? Hint: use `scipy.stats.norm.cdf`.

`scipy.stats` contains objects that represent analytic distributions

In [34]:
import scipy.stats

For example scipy.stats.norm represents a normal distribution.

In [35]:
mu = 178
sigma = 7.7
dist = scipy.stats.norm(loc=mu, scale=sigma)
type(dist)

scipy.stats._distn_infrastructure.rv_frozen

scipy.stats._distn_infrastructure.rv_frozen

A "frozen random variable" can compute its mean and standard deviation.

In [36]:
dist.mean(), dist.std()

(178.0, 7.7)

It can also evaluate its CDF.  How many people are below the mean by more than one standard deviation?  About 16%

In [37]:
dist.cdf(mu - sigma)

0.1586552539314574

How many people are between 5'10" and 6'1"?

In [38]:
# Convert inches to cm (I found this formula on stackoverflow to convert ft multiple
#the feet by 12 and sum 10.

min_height = 2.54 * (5*12+10)
max_height = 2.54 * (6*12+1)

print(('In cm, '), + (round(min_height,2)))
print(('In cm, '), + (round(max_height,2)))


In cm,  177.8
In cm,  185.42


In [39]:
# evaluate the CDF.
#A cumulative distribution function (CDF) is defined as a function F(x) 
#that is the probability that a random variable c, from a particular distribution,
#is less than x.

#calculated the % of men population that is at the 5'10'' height (49%)
min = dist.cdf(min_height)*100
print(round(min))

#calculated the % of men population that is at the 6'1'' height (83%)
max = dist.cdf(max_height)*100
print(round(max))

#calculated the % of men population that is in this range (5'10'' to 6'1'') (34%)
percentage = max - min
print(round(percentage))

49
83
34


About 34% of the men population would be able to join Blue Man Group.

Chapter 5
Exercise 5.2

**Exercise:** To get a feel for the Pareto distribution, let’s see how different the world would be if the distribution of human height were Pareto. With the parameters xm = 1 m and α = 1.7, we get a distribution with a reasonable minimum, 1 m, and median, 1.5 m.

Plot this distribution. What is the mean human height in Pareto world? What fraction of the population is shorter than the mean? If there are 7 billion people in Pareto world, how many do we expect to be taller than 1 km? How tall do we expect the tallest person to be?

`scipy.stats.pareto` represents a pareto distribution.  In Pareto world, the distribution of human heights has parameters alpha=1.7 and xmin=1 meter.  So the shortest person is 100 cm and the median is 150.

In [40]:
alpha = 1.7
xmin = 1  # meter
dist = scipy.stats.pareto(b=alpha, scale=xmin)
dist.median()

1.5034066538560549

What is the mean height in Pareto world?

In [41]:
# here I calculated the mean in meters

#calculate = (alpha - xmin)/alpha-1)
#calculate = (1.7/0.7)
#print(calculate)

mean_height = dist.mean()
print(('In meters, '), + (round(mean_height,2)))

In meters,  2.43


What fraction of people are shorter than the mean?

In [42]:
#here I used the CDF to calculate the % that are shorter than the 
#mean calculated above.

fraction_shorter = dist.cdf(mean_height)*100
print(round(fraction_shorter,2))

77.87


Out of 7 billion people, how many do we expect to be taller than 1 km?  You could use dist.cdf  or dist.sf.

In [43]:
#since xmin = 1 meter, I need to convert 1km to meters = 1000 meters

taller_than_1km = (1 - dist.cdf(1000)) * 7000000000
print(taller_than_1km)

55602.976430479954


About 55,602 people would be taller than 1km.

How tall do we expect the tallest person to be?

In [44]:
#Using ppf (percent point function), which is the inverse CDF. 

dist.ppf(1 - 1 / 7000000000)/1000

618.3496106759505

The tallest person would be about 618km.

Chapter 6
Exercise 6.1

The distribution of income is famously skewed to the right. In this exercise, we’ll measure how strong that skew is.
The Current Population Survey (CPS) is a joint effort of the Bureau of Labor Statistics and the Census Bureau to study income and related variables. Data collected in 2013 is available from http://www.census.gov/hhes/www/cpstables/032013/hhinc/toc.htm. I downloaded `hinc06.xls`, which is an Excel spreadsheet with information about household income, and converted it to `hinc06.csv`, a CSV file you will find in the repository for this book. You will also find `hinc2.py`, which reads this file and transforms the data.

The dataset is in the form of a series of income ranges and the number of respondents who fell in each range. The lowest range includes respondents who reported annual household income “Under \$5000.” The highest range includes respondents who made “\$250,000 or more.”

To estimate mean and other statistics from these data, we have to make some assumptions about the lower and upper bounds, and how the values are distributed in each range. `hinc2.py` provides `InterpolateSample`, which shows one way to model this data. It takes a `DataFrame` with a column, `income`, that contains the upper bound of each range, and `freq`, which contains the number of respondents in each frame.

It also takes `log_upper`, which is an assumed upper bound on the highest range, expressed in `log10` dollars. The default value, `log_upper=6.0` represents the assumption that the largest income among the respondents is $10^6$, or one million dollars.

`InterpolateSample` generates a pseudo-sample; that is, a sample of household incomes that yields the same number of respondents in each range as the actual data. It assumes that incomes in each range are equally spaced on a `log10` scale.

In [45]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [46]:
import numpy as np

import thinkstats2
import thinkplot

In [47]:
def InterpolateSample(df, log_upper=6.0):
    """Makes a sample of log10 household income.

    Assumes that log10 income is uniform in each range.

    df: DataFrame with columns income and freq
    log_upper: log10 of the assumed upper bound for the highest range

    returns: NumPy array of log10 household income
    """
    # compute the log10 of the upper bound for each range
    df['log_upper'] = np.log10(df.income)

    # get the lower bounds by shifting the upper bound and filling in
    # the first element
    df['log_lower'] = df.log_upper.shift(1)
    df.loc[0, 'log_lower'] = 3.0

    # plug in a value for the unknown upper bound of the highest range
    df.loc[41, 'log_upper'] = log_upper
    
    # use the freq column to generate the right number of values in
    # each range
    arrays = []
    for _, row in df.iterrows():
        vals = np.linspace(row.log_lower, row.log_upper, int(row.freq))
        arrays.append(vals)

    # collect the arrays into a single sample
    log_sample = np.concatenate(arrays)
    return log_sample


In [48]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/hinc.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/hinc06.csv")

In [73]:
import hinc
income_df = hinc.ReadData()

In [90]:
log_sample = InterpolateSample(income_df, log_upper=6.0)

In [91]:
sample = np.power(10, log_sample)

Compute the median, mean, skewness and Pearson’s skewness of the resulting sample. What fraction of households report a taxable income below the mean? How do the results depend on the assumed upper bound?

In [92]:
#Here I calculated the median of the sample

def Median(xs):
    cdf = thinkstats2.Cdf(xs)
    return cdf.Value(0.5)

Median(sample)

51226.45447894046

In [93]:
def RawMoment(xs, k):
    return sum(x**k for x in xs) / len(xs)

In [94]:
#Here I calculated the mean of the sample. I created a variable called mean_sample
#so I could use at the fraction less then the mean below
def Mean(xs):
    return RawMoment(xs, 1)

mean_sample = Mean(sample)
print(mean_sample)

74278.70753118733


In [95]:
#here I calculated the skewness 
def StandardizedMoment(xs, k):
    var = CentralMoment(xs, 2)
    std = np.sqrt(var)
    return CentralMoment(xs, k) / std**k

def Skewness(xs):
    return StandardizedMoment(xs, 3)

Skewness(sample)

4.949920244429583

The 4.94 skewness is positive and indicate its skewed right (longer tail)

In [96]:
#calculated the Pearson Median Skewness
def PearsonMedianSkewness(xs):
    median = Median(xs)
    mean = RawMoment(xs, 1)
    var = CentralMoment(xs, 2)
    std = np.sqrt(var)
    gp = 3 * (mean - median) / std
    return gp

PearsonMedianSkewness(sample)

0.7361258019141782

In [97]:
#verified the fraction of population that income is less than the mean in %
fraction_less_mean = cdf.Prob(mean_sample)*100
print(round(fraction_less_mean,2))

66.0


66% of households report a taxable income below the mean. Since the dataset is in a form of a series of income ranges, if I change the upper limit, the mean will increase. I did a test and changing the log_upper from 6 (one million) to 7(10 million), that would bring the mean to 124,267 and the fraction of population below that to 85%. The skewness increases to 11.60 which will make the tail longer to the right.

All of this is based on an assumption that the highest income is one million dollars, but that's certainly not correct.  What happens to the skew if the upper bound is 10 million?



Without better information about the top of this distribution, we can't say much about the skewness of the distribution.